## 1. LangSmith Setup

#### Add these entries to your `.env` file
```bash
# LangSmith Environment Configuration
LANGCHAIN_PROJECT="Langsmith_Eval"
LANGCHAIN_TRACING_V2=true
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY=<your_langsmith_api_key_here>


In [2]:
# 2. Install Required Packages
# %pip install langchain langsmith openai python-dotenv

In [8]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langsmith import traceable # Need to enable tracing on LangSmith


# --- Setup ---
load_dotenv(override=True)
my_api_key = os.getenv("OPENAI_API_KEY")

# Initialize model
llm = ChatOpenAI(model="gpt-5-nano", api_key=my_api_key)


# --- Function under test ---
@traceable # Enable tracing for this function on LangSmith - Only one line change
def ask_question(user_prompt) -> str:
    """LLM function under test."""
    # If LangSmith passes a dict, extract the actual string
    if isinstance(user_prompt, dict):
        user_prompt = user_prompt.get("input", "")

    system_msg = "You are a helpful assistant. Provide answer in one line."
    response = llm.invoke(
        [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_prompt},
        ]
    )
    
    return response.content

In [9]:
ask_question("Where is the mueseum Louvre?")

'In Paris, France, on the Right Bank of the Seine at 99 Rue de Rivoli (75001).'

#### You should see your project being tracked at [https://smith.langchain.com/](https://smith.langchain.com/)

## 2. LangSmith DataSets

In [1]:
from langsmith import Client


client = Client()
dataset = client.create_dataset(dataset_name="LangSmith-QA-Dataset")


examples = [
{"inputs": {"question": "What is LangChain?"}, "outputs": {"answer": "A framework to build LLM-powered apps."}},
{"inputs": {"question": "What is LangSmith?"}, "outputs": {"answer": "A platform for tracing and evaluating LLM applications."}},
]


for ex in examples:
    client.create_example(inputs=ex["inputs"], outputs=ex["outputs"], dataset_id=dataset.id)

print(" Dataset created:", dataset.name)

 Dataset created: LangSmith-QA-Dataset
